# House Prices - Advanced Regression Techniques
(Preços de casas - Técnicas Avançadas de Regressão)

<img src="https://storage.googleapis.com/kaggle-media/competitions/House%20Prices/kaggle_5407_media_housesbanner.png" width=800>


- Vamos utilizar o [dataset disponível no Kaggle](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview)
    - É um dataset de **competição**
    - Temos uma base com a **descrição de cada uma das colunas (data_description.txt)**
- **Etapas do Projeto:**
    - O projeto será realizado aplicando submissões no kaggle a medida que o tratamento dos dados forem tomando forma, isso possibilita verificar o desempenho dos modelos preditivos a modificações específicas neste tratamento.

In [151]:
# importando o pandas
import pandas as pd

In [152]:
# Importando as bases de treino e teste do checkpoint 1
base_train = pd.read_csv("train_1.csv")
base_teste = pd.read_csv("test_1.csv")         

# Continuar análise da base

### Tratar colunas categóricas

In [153]:
# Filtrar colunas categóricas
col_cat = base_train.select_dtypes(include=object).columns

In [154]:
base_train[col_cat]

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,SBrkr,Gd,Typ,-1,Attchd,RFn,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,SBrkr,TA,Typ,TA,Attchd,RFn,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,SBrkr,TA,Typ,TA,Attchd,RFn,TA,Y,WD,Normal
1456,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,...,SBrkr,TA,Min1,TA,Attchd,Unf,TA,Y,WD,Normal
1457,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,...,SBrkr,Gd,Typ,Gd,Attchd,RFn,TA,Y,WD,Normal
1458,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,...,FuseA,Gd,Typ,-1,Attchd,Unf,TA,Y,WD,Normal


### Colunas com apenas 2 valores distintos
- Nestes casos podemos utilizar a binarização
- Também podemos agrupar valores de outras colunas que apresentem um valor muito predominante
    - Aqui foi considerado que se a coluna apresentasse um valor com mais de 80% de frequencia, os demais valores poderiam       ser agrupados em um único

In [155]:
# Visão de como funciona essa etapa
# Percentual de frequenci de cada valor nessa coluna específica
# Se o valor de maior frequencia for maior que 80% os demais serão agrupados em uma só categoria pois possuem baixa frequencia
# Esse procedimento não seria interessante se o problema do negócio fosse a identificação de anomalia, como identificar fraudes
(base_train.groupby('MSZoning')['SalePrice'].count()/base_train.shape[0])*100

MSZoning
C (all)     0.684932
FV          4.452055
RH          1.095890
RL         78.835616
RM         14.931507
Name: SalePrice, dtype: float64

In [156]:
# Filtrar colunas com valor predominante superior a 80% de frequencia e realizar binarização
colunas_binary = []
for i in base_train.columns:
     if ((base_train.groupby(i)['SalePrice']
          .count().max())/base_train.shape[0])>0.8:
            colunas_binary.append(i)
colunas_binary

['Street',
 'LandContour',
 'Utilities',
 'LandSlope',
 'Condition1',
 'Condition2',
 'BldgType',
 'RoofMatl',
 'ExterCond',
 'BsmtCond',
 'BsmtFinType2',
 'BsmtFinSF2',
 'Heating',
 'CentralAir',
 'Electrical',
 'LowQualFinSF',
 'BsmtHalfBath',
 'KitchenAbvGr',
 'Functional',
 'GarageQual',
 'PavedDrive',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'MiscVal',
 'SaleType',
 'SaleCondition']

In [157]:
# Realiza a binarização
for i in colunas_binary:
    base_train[i] = base_train[i].apply(
        lambda x: 0 if x == base_train[i].value_counts()
        .sort_values(ascending=False).index[0] else 1)
    base_train[i] = base_train[i].astype(int)
    
    #Realizar na base de teste
    base_teste[i] = base_teste[i].apply(
        lambda x: 0 if x == base_teste[i].value_counts()
        .sort_values(ascending=False).index[0] else 1)
    base_teste[i] = base_teste[i].astype(int)    

In [158]:
# Observar colunas restantes
col_rest = base_train.select_dtypes(include=object).columns
col_rest

Index(['MSZoning', 'LotShape', 'LotConfig', 'Neighborhood', 'HouseStyle',
       'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC',
       'KitchenQual', 'FireplaceQu', 'GarageType', 'GarageFinish'],
      dtype='object')

### Dividir entre colunas Ordinais e não Ordinais
#### Ordinais:  'ExterQual','HeatingQC','KitchenQual','FireplaceQu','GarageFinish','LotShape','BsmtQual','BsmtExposure','BsmtFinType1','GarageType'.

In [159]:
col_ord = ['ExterQual','HeatingQC','KitchenQual','FireplaceQu','GarageFinish','LotShape','BsmtQual','BsmtExposure','BsmtFinType1','GarageType']
base_train[col_ord]

,ExterQual,HeatingQC,KitchenQual,FireplaceQu,GarageFinish,LotShape,BsmtQual,BsmtExposure,BsmtFinType1,GarageType
0,Gd,Ex,Gd,-1,RFn,Reg,Gd,No,GLQ,Attchd
1,TA,Ex,TA,TA,RFn,Reg,Gd,Gd,ALQ,Attchd
2,Gd,Ex,Gd,TA,RFn,IR1,Gd,Mn,GLQ,Attchd
3,TA,Gd,Gd,Gd,Unf,IR1,TA,No,ALQ,Detchd
4,Gd,Ex,Gd,TA,RFn,IR1,Gd,Av,GLQ,Attchd
...,...,...,...,...,...,...,...,...,...,...
1455,TA,Ex,TA,TA,RFn,Reg,Gd,No,Unf,Attchd
1456,TA,TA,TA,TA,Unf,Reg,Gd,No,ALQ,Attchd
1457,Ex,Ex,Gd,Gd,RFn,Reg,TA,No,GLQ,Attchd
1458,TA,Gd,Gd,-1,Unf,Reg,TA,Mn,GLQ,Attchd


### Realizar o OrdinalEncoder nas colunas ordinais

In [160]:
# Importando OrdinalEncoder
from sklearn.preprocessing import OrdinalEncoder

In [161]:
base_train[col_ord].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ExterQual     1460 non-null   object
 1   HeatingQC     1460 non-null   object
 2   KitchenQual   1460 non-null   object
 3   FireplaceQu   1460 non-null   object
 4   GarageFinish  1460 non-null   object
 5   LotShape      1460 non-null   object
 6   BsmtQual      1460 non-null   object
 7   BsmtExposure  1460 non-null   object
 8   BsmtFinType1  1460 non-null   object
 9   GarageType    1460 non-null   object
dtypes: object(10)
memory usage: 114.2+ KB


In [162]:
base_train.GarageType.value_counts()

GarageType
Attchd     870
Detchd     387
BuiltIn     88
-1          81
Basment     19
CarPort      9
2Types       6
Name: count, dtype: int64

In [163]:
# Devo passar manualmente os valores ordenados de cada coluna (indicado na descrição dos dados)
col_ord = ['ExterQual','HeatingQC','KitchenQual','FireplaceQu','GarageFinish',
           'LotShape','BsmtQual','BsmtExposure','BsmtFinType1','GarageType']
cat1 = ['Fa','TA','Gd','Ex']
cat2 = ['Po','Fa','TA','Gd','Ex']
cat3 = ['Fa','TA','Gd','Ex']
cat4 = ['-1','Po','Fa','TA','Gd','Ex']
cat5 = ['-1','Unf','RFn','Fin']
cat6 = ['IR3','IR2','IR1','Reg']
cat7 = ['-1','Fa','TA','Gd','Ex']
cat8 = ['-1','No','Mn','Av','Gd']
cat9 = ['-1','Unf','LwQ','Rec','BLQ','ALQ','GLQ']
cat10 = ['-1','Detchd','CarPort','BuiltIn','Basment','Attchd','2Types']

categories = ['']
for i in range(10): 
    categorias = f'cat{i+1}' 
    # Criando o objeto e ajustando as categorias
    oe = OrdinalEncoder(categories=[globals()[categorias]])
    # Transformando os dados diretamente
    base_train[col_ord[i]] = oe.fit_transform(base_train[[col_ord[i]]])  


In [164]:
# Agora estão contendo valores numéricos ordenados
base_train[col_ord].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ExterQual     1460 non-null   float64
 1   HeatingQC     1460 non-null   float64
 2   KitchenQual   1460 non-null   float64
 3   FireplaceQu   1460 non-null   float64
 4   GarageFinish  1460 non-null   float64
 5   LotShape      1460 non-null   float64
 6   BsmtQual      1460 non-null   float64
 7   BsmtExposure  1460 non-null   float64
 8   BsmtFinType1  1460 non-null   float64
 9   GarageType    1460 non-null   float64
dtypes: float64(10)
memory usage: 114.2 KB


In [165]:
# Colunas restantes
col_rest = base_train.select_dtypes(include=object).columns
col_rest

Index(['MSZoning', 'LotConfig', 'Neighborhood', 'HouseStyle', 'RoofStyle',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation'],
      dtype='object')

**Realizar mesmo procedimento na base de teste**

In [166]:
base_teste[col_ord].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ExterQual     1459 non-null   object
 1   HeatingQC     1459 non-null   object
 2   KitchenQual   1459 non-null   object
 3   FireplaceQu   1459 non-null   object
 4   GarageFinish  1459 non-null   object
 5   LotShape      1459 non-null   object
 6   BsmtQual      1459 non-null   object
 7   BsmtExposure  1459 non-null   object
 8   BsmtFinType1  1459 non-null   object
 9   GarageType    1459 non-null   object
dtypes: object(10)
memory usage: 114.1+ KB


In [167]:
base_teste.KitchenQual.value_counts()

KitchenQual
TA    758
Gd    565
Ex    105
Fa     31
Name: count, dtype: int64

In [168]:
# Devo passar manualmente os valores ordenados de cada coluna (indicado na descrição dos dados)
col_ord = ['ExterQual','HeatingQC','KitchenQual','FireplaceQu','GarageFinish',
           'LotShape','BsmtQual','BsmtExposure','BsmtFinType1','GarageType']
cat1 = ['Fa','TA','Gd','Ex']
cat2 = ['Po','Fa','TA','Gd','Ex']
cat3 = ['Fa','TA','Gd','Ex']
cat4 = ['-1','Po','Fa','TA','Gd','Ex']
cat5 = ['-1','Unf','RFn','Fin']
cat6 = ['IR3','IR2','IR1','Reg']
cat7 = ['-1','Fa','TA','Gd','Ex']
cat8 = ['-1','No','Mn','Av','Gd']
cat9 = ['-1','Unf','LwQ','Rec','BLQ','ALQ','GLQ']
cat10 = ['-1','Detchd','CarPort','BuiltIn','Basment','Attchd','2Types']

categories = ['']
for i in range(10): 
    categorias = f'cat{i+1}' 
    # Criando o objeto e ajustando as categorias
    oe = OrdinalEncoder(categories=[globals()[categorias]])
    # Transformando os dados diretamente
    base_teste[col_ord[i]] = oe.fit_transform(base_teste[[col_ord[i]]])  

In [169]:
# Agora estão contendo valores numéricos ordenados
base_teste[col_ord].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ExterQual     1459 non-null   float64
 1   HeatingQC     1459 non-null   float64
 2   KitchenQual   1459 non-null   float64
 3   FireplaceQu   1459 non-null   float64
 4   GarageFinish  1459 non-null   float64
 5   LotShape      1459 non-null   float64
 6   BsmtQual      1459 non-null   float64
 7   BsmtExposure  1459 non-null   float64
 8   BsmtFinType1  1459 non-null   float64
 9   GarageType    1459 non-null   float64
dtypes: float64(10)
memory usage: 114.1 KB


## Agrupar valores pouco frequentes
#### Assim como foi realizada na binarização, posso reduzir a quantidade de classes observando a frequencia delas, isso rerá realizado nas colunas restantes. 
#### Aqui no entanto, serão mantidas os dois valores mais frequentes e agrupados os demais em uma única outra categoria

**IMPORTANTE: Realizar este procedimento apenas nas colunas não ordinais**
- Considerações:
    - O ideal seria agrupar esses valores de forma manual, de acordo com o negócio
    - Por exemplo: A coluna de vizinhança poderia ser agrupada por locais que são próximos
    - Esse agrupamento reduz a quantidade de colunas que serão criadas no processo do One Hot Encoder

In [170]:
# Selecionar as colunas que se encaixam nessas especificações
colunas_tres = []
for i in col_rest:
     if (((base_train.groupby(i)['SalePrice'].count()
           .sort_values(ascending=False)
           /base_train.shape[0])[0]
           +(base_train.groupby(i)['SalePrice']
           .count().sort_values(ascending=False)
            /base_train.shape[0])[1])>0.8):
        colunas_tres.append(i)
colunas_tres

['MSZoning',
 'LotConfig',
 'HouseStyle',
 'RoofStyle',
 'MasVnrType',
 'Foundation']

In [171]:
# Observar cardinalidade dessas colunas antes do tratamento
base_train[col_rest].nunique()

MSZoning         5
LotConfig        5
Neighborhood    25
HouseStyle       8
RoofStyle        6
Exterior1st     15
Exterior2nd     16
MasVnrType       4
Foundation       6
dtype: int64

In [172]:
# Com exceção das duas classes de maior frequencia
# Transformar todas outras classes em uma única 'outro'
# Trasformar base de treino e teste
for i in col_rest:
    # Base de teste
    base_teste[i] = base_teste[i].apply(lambda x: 'Outro' 
    if (x != base_train.groupby(i)['SalePrice'].count()
    .sort_values(ascending=False).index[0])&(x != base_train.groupby(i)
    ['SalePrice'].count().sort_values(ascending=False).index[1]) else x)
    # Base de treino
    base_train[i] = base_train[i].apply(lambda x: 'Outro' 
    if (x != base_train.groupby(i)['SalePrice'].count()
    .sort_values(ascending=False).index[0])&(x != base_train.groupby(i)
    ['SalePrice'].count().sort_values(ascending=False).index[1]) else x)

In [173]:
# Observar cardinalidade depois do tratamento
print(base_train[col_rest].nunique())
print(base_teste[col_rest].nunique())

MSZoning        3
LotConfig       3
Neighborhood    3
HouseStyle      3
RoofStyle       3
Exterior1st     3
Exterior2nd     3
MasVnrType      3
Foundation      3
dtype: int64
MSZoning        3
LotConfig       3
Neighborhood    3
HouseStyle      3
RoofStyle       3
Exterior1st     3
Exterior2nd     3
MasVnrType      3
Foundation      3
dtype: int64


### Agora posso realizar o processo de One Hot Encoder
**Esse último procedimento foi importante para reduzir a quantidade de colunas que serão geradas agora no One Hot Encoder**

### One Hot Encoder

In [174]:
# Importando
from sklearn.preprocessing import OneHotEncoder

In [175]:
# Utilizando o OneHotEncoder
ohe = OneHotEncoder()

In [176]:
# Fazendo o fit com os dados
# Posso considerar mais de uma coluna no ohe
ohe = ohe.fit(base_train[col_rest])

In [177]:
# Fazendo a transformação da coluna Embarked
ohe.transform(base_train[col_rest]).toarray()

array([[0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [178]:
# Podemos transformar esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(base_train[col_rest]).toarray())

In [179]:
# Agora podemos utilizar o categories_ para nomear essas colunas
ohe_df.columns = ohe.get_feature_names_out()

**Podemos notar que as 9 colunas se tornaram 27**

In [180]:
# Precisamos agora dropar as colunas antigas e concatenar essa base na base original
base_train = base_train.drop(col_rest,axis=1)

In [181]:
# Concatenar
base_train = pd.concat([base_train,ohe_df], axis=1)
base_train.shape

(1460, 91)

In [182]:
base_train.head()

,Id,MSSubClass,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LandSlope,Condition1,...,Exterior1st_VinylSd,Exterior2nd_MetalSd,Exterior2nd_Outro,Exterior2nd_VinylSd,MasVnrType_-1,MasVnrType_BrkFace,MasVnrType_Outro,Foundation_CBlock,Foundation_Outro,Foundation_PConc
0,1,60,65.0,8450,0,3.0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,20,80.0,9600,0,3.0,0,0,0,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,3,60,68.0,11250,0,2.0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,4,70,60.0,9550,0,2.0,0,0,0,0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,5,60,84.0,14260,0,2.0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [183]:
# Observar se restaram colunas de texto
print(base_train.select_dtypes(object).columns)

Index([], dtype='object')


### Repetir procedimento para base de teste

In [184]:
# Utilizando o OneHotEncoder
ohe = OneHotEncoder()

In [185]:
# Fazendo o fit com os dados
# Posso considerar mais de uma coluna no ohe
ohe = ohe.fit(base_teste[col_rest])

In [186]:
# Fazendo a transformação da coluna Embarked
ohe.transform(base_teste[col_rest]).toarray()

array([[1., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.]])

In [187]:
# Podemos transformar esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(base_teste[col_rest]).toarray())

In [188]:
# Agora podemos utilizar o categories_ para nomear essas colunas
ohe_df.columns = ohe.get_feature_names_out()

In [189]:
# Precisamos agora dropar as colunas antigas e concatenar essa base na base original
base_teste = base_teste.drop(col_rest,axis=1)

In [190]:
# Concatenar
base_teste = pd.concat([base_teste,ohe_df], axis=1)
base_teste.shape

(1459, 90)

In [191]:
base_teste.head()

,Id,MSSubClass,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LandSlope,Condition1,...,Exterior1st_VinylSd,Exterior2nd_MetalSd,Exterior2nd_Outro,Exterior2nd_VinylSd,MasVnrType_-1,MasVnrType_BrkFace,MasVnrType_Outro,Foundation_CBlock,Foundation_Outro,Foundation_PConc
0,1461,20,80.0,11622,0,3.0,0,0,0,1,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1462,20,81.0,14267,0,2.0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1463,60,74.0,13830,0,2.0,0,0,0,0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1464,60,78.0,9978,0,2.0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1465,120,43.0,5005,0,2.0,1,0,0,0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [192]:
# Observar se restaram colunas de texto
print(base_teste.select_dtypes(object).columns)

Index([], dtype='object')


## Checkpoint 3
- Terceira base para observar o desempenho nos modelos que forem escolhidos

In [195]:
# Exportando a base de treino e teste
base_train.to_csv('train_3.csv',index=False)
base_teste.to_csv('test_3.csv',index=False)